In [ ]:
##SQL Functions and Subqueries##

```postgresql
query1 = "DELETE FROM read.book
WHERE name IN (" + "%s"*10 + "+";"

query2 =  "DELETE FROM read.book
WHERE id IN (%s, %s)"
```

In [ ]:
```postgresql

SELECT * FROM read.book WHERE status='complete' AND start_read_date >= %s;
````

conn.execute(query, ())

conn =
cursor

query = ```DELETE FROM read.book WHERE id= %s```


```
-- block table from updated or anything
for id selected_ids:
    cursor.execute(query, id)
    
-- unblock

# Joining Tables

- Multiple tables are related.
- Usually, we start with a single table.
- The main issue we try to avoid is redundant information.
- The technique used to prevent redundant information is `Normalization`

## Normalization

> Is a way of structuring your database so as to avoid expressing redundant information.

### Benefits

- Easier to understand
- Easier to extend
- Protected from:
  - Insertion anomalies
  - Update anomalies
  - Deletion anomalies


#### Normal Forms

- The three forms in normalization are `1NF`, `2NF` and `3NF`.
- The inventor of Rational Models: `Frank Codd`, he came up with the first three normal forms listed above.


### 1NF

- Mixing data types within the same column violates 1NF
- A table without a primary key violates 1NF
  - Our table has a composite key of columns `emp_id`, `salary`, `dept`, `city`
- Each table cell should contain a single value.
  - Our table has the column `name` that violates this rule.


### 2NF

- The table should be in 1NF

> Each non-key attribute must depend on the entier primary key.


### 3NF

- The table should be in 2NF

> A non-key attribute shouldn't depend on a non-key attribute. It should rather depend on the whole key, and nothing but the key.

```text
skill -> seniority
----------
1,2,3,4 -> Beginner
5,6,7 -> Intermidiate
8,9,10 -> Advanced

```


## Exercise

- Query your database to the the following information
  - Return `title`, `first_name`, `last_name`, `seniority_cat` for all employees
  - Return `name`, `salary`, `dept`, `skill_level` for all employee with salary greater than 500. Where `name` is a combination of `title`, `first_name` and `last_name`.
  - Practice more queries
- Restructure the database of the my read app by applying normalization.


### Solution

- 1.

```postgresql
SELECT title, first_name, last_name, seniority_cat
FROM employee, skill, seniority
WHERE employee.emp_id=skill.emp_id
    AND skill.skill_level = seniority.skill_level;
```

- 2

```postgresql

SELECT e.title || ' ' || e.first_name || ' ' || e.last_name AS name, er.salary, d.dept, s.skill_level
FROM employee e , emp_role er, department d, skill s
WHERE e.emp_id=er.emp_id
    AND er.dept_id = d.dept_id
    AND e.emp_id=s.emp_id
    AND er.salary > 500;

```


## TABLE JOINS

- Two ways of joining using the JOIN clause:
  - `JOIN .... ON`
  - `JOIN ... USING`

### Questions

- What are the two important and commonly used constraints for table join to happen securely?
- What could be the properties or constraints enforced on a PRIMARY KEY
- What is a FOREIGN KEY
- What are the differencs between PRIMARY KEY and FOREIGN KEY
  - Unlike the PRIMARY KEY, the FOREIGN KEY can be null or contain duplicates.


## Joins Types

- We have these different types of JOINs
  - Inner
  - Outer
  - Cross
  - Self


#### INNER JOIN or JOIN

> Returns rows from both tables where matching values are found in the joined columns of both table.

- Intersection

```postgresql
-- Syntax
SELECT <column-list>
FROM <pk-table> JOIN <fk-table>
ON <pk-table>.key-column = <fk-table>.fk-column
[WHERE ...];

-- Query
SELECT *
FROM employee JOIN emp_role
ON employee.emp_id = emp_role.emp_id;

-- insert
INSERT INTO employee (title, last_name)
VALUES
    ('Dr.', 'Brown');

-- USING
-----  Both foreign key and primary key have the same name
------ The common operator used on the ON clause is `=`

SELECT *
FROM employee JOIN emp_role
USING(emp_id);
```

**When to Use**

- When you want to return only records having pair on both sides(Intersection)


## OUTER JOINS

- `LEFT OUTER JOIN` or `LEFT JOIN`
- `RIGHT OUTER JOIN` or `RIGHT JOIN`

### LEFT JOIN

> Returns rows from the left table, plus rows that match values in the joined column from the right table. If the left table doesn't have any match in the right table, the results shows no values from the right table by displaying blank rows or wahtever NULL was set to.

- The left table is mandatory
- The right table is optional

```postgresql
SELECT *
FROM employee LEFT JOIN emp_role
USING(emp_id)
WHERE emp_role.emp_id is NULL;

```

### RIGHT JOIN

> Returns rows from the right table, plus rows that match values in the joined column from the left table. If the right table doesn't have any match in the left table, the results shows no values from the left table by displaying blank rows or whatever NULL was set to.

- The right table is mandatory
- The left table is optional

```postgresql
SELECT *
FROM employee RIGHT JOIN emp_role
USING(emp_id);
```


### FULL OUTER JOIN or FULL JOIN

- Symmetric difference.
  > Returns every row from both tables. If there is no match in the joined column in any of the tables, a blank row is displayed.

```postgresql
SELECT *
FROM emp_role FULL JOIN employee
ON emp_role.emp_id + 1  = employee.emp_id + 2
WHERE emp_role.emp_id IS NULL
    OR employee.emp_id IS NULL;

```

**When to use**

- To find the symmetric difference between two or more tables
- To see all rows from all tables, regardless of whether any match.
  - To visualize the degree to which the tables share matching values


## CROSS JOIN

> Returns every possible combinations of rows from all tables

- Cartesian product.
  - If you have `table_a` with `n` rows, and `table_b` with `m` rows, then CROSS JOINING these tables will return `n x m` number of rows.
- It doesn't require `ON` or `USING`.

```postgresql
SELECT last_name, dept
FROM department CROSS JOIN employee;
```

**When not to use**

- Not good for large tables due to the size of the output

**When to use**

- shirts
  - table a: contain different model of shirts
    - modal a
    - model b
  - table b: contain different colors
    - red
    - yellow
    - white
  - Output
    - model a, red
    - model a, yellow
    - model a, white
    - model b, red
    - model b, yellow
    - model b, white
- Wharehouse
  - product
    - shoes
  - shops
    - shop a
    - shop b
    - shop c
  - Output
    - shoes, shop a
    - shoes, shop b
    - shoes, shop c


## SELF JOINS

- It's just a way of naming a specific inner join.
  - It is a type of inner join where a table is joined to itself.

```postgresql
-- create
CREATE TABLE church_hierarchy (
    pos_id SERIAL,
    pos_desc VARCHAR(100),
    supervisor_id INT
);

- insert

INSERT INTO church_hierarchy(pos_desc, supervisor_id)
VALUES
    ('God', null),
    ('Pope', 1),
    ('Cardinal', 2),
    ('Biship', 3),
    ('Priest', 4);

-- HIERARCHY
God -> Pope
Pope -> Cardinal
Cardinal -> Bishop
Bishop -> Priest

-- SELF JOIN

SELECT a.pos_desc Supervisor, b.pos_desc Worker
FROM church_hierarchy a
JOIN church_hierarchy b
ON a.pos_id = b.supervisor_id;
```


## Exercise

```postgresql
-- BASIC SELECT
SELECT e.title || ' ' || e.first_name || ' ' || e.last_name AS name, er.salary, d.dept, s.skill_level
FROM employee e , emp_role er, department d, skill s
WHERE e.emp_id=er.emp_id
    AND er.dept_id = d.dept_id
    AND e.emp_id=s.emp_id
    AND er.salary > 500;

-- INNER JOIN
SELECT <column-list>
FROM <pk-table>
    JOIN <pk-fk-table> ON ...
        JOIN <fk-table> ON ..

-- SOLUTION
SELECT title || ' ' || first_name || ' ' || last_name AS name, salary, dept, skill_level
FROM employee
    JOIN skill USING(emp_id)
        JOIN emp_role USING (emp_id)
            JOIN department USING(dept_id);

```


## Exercise

- Federica exercises: https://classroom.github.com/a/UO_S7vNX
- After having restructured your database for the my read app, go ahead and refactor the code to use your restructured database.
  - _SELF-STUDY_
    - postgresql foreign key and create your foreign key with appropriate actions so that deletions and updates can be handled properly. I recommend the official doc: https://www.postgresql.org/docs/current/ddl-constraints.html#DDL-CONSTRAINTS-FK


```psql
\pset null
\pset null <value>
```


# Postgresql Advanced  Data Types

## Numeric Types
- Integer types
- Serial types
- arbitrary pecision numbers
    - numeric / decimal
- Floating point types
    - float
    - real
    - double precision
    - infinity and -infinity


### Monetary types
- This handles monetary values
- We can set the currency and scale of the decimal point with `lc_monetary`
- With:

```postgresql
SET lc_monetary = <value>

-- value comes from
SELECT collname
FROM pg_collation;
```

### Date/Time types
- SQL, DMY
- SQL, MYD
- ISO, DMY # default

```postgresql
-- change date output format
SET DATESTYLE = <value>;
```

#### Interval data type
- Helps to store and minipulate period of time in years, months, hours, minutes, seconds.

- We can use the following case-insensitive time periods
    - Year(s) -> `y`
    - Month(s) -> What will this be?
    - week(s) -> `w`
    - Day(s) -> `d`
    - Minute(s) -> `m`
    - Second(s) -> `s`.

- User
    - deletion_request: current_date
    - deleted: False
    
- conf
    - deletion_interval: INTERVAL -> '3 days'
    
- cron jobs
    - deletion_request + deletion_interval = current_date
        - deletion_request: NULL
        - deleted: True
        
    - Separated our cron job to a microservices.
    - When a user request to be deleted
        - schedule a cron job with that user id.
        - cron job is deleted.
    
#### Boolean Type
- `true`, `false`

```postgresql
SELECT 4::BOOL; -- t
SELECT 0::BOOL; -- F
SELECT -1::BOOL; -- error
SELECT 4.66::BOOL; -- error
SELECT '0'::BOOL; -- f
SELECT '1'::BOOL; -- t
SELECT '3'::BOOL; -- error
SELECT true; -- t
SELECT false; -- f
SELECT 'true'::BOOL; -- t
SELECT 'false'::BOOL; -- f
SELECT 'falsee'::BOOL; -- error
SELECT 't'::BOOL; -- t
SELECT 'f'::BOOL; -- f
SELECT 'n'::BOOL; -- f
SELECT 'y'::BOOL; -- t
SELECT 'no'::BOOL; -- f
SELECT 'yes'::BOOL; -- t

```

#### Enumerated type
- Has the syntax

```postgresql
CREATE TYPE <type-name> AS ENUM(...);
```

### Network address type
- Store and manipulate network addresses lile `IPv4`, `IPv6`, `MAC`.
- The data types that we can use with these data are ;
    - `cidr` -> IPv4 and IPv6
    - `inet` -> IPv4 and IPv6 host and networks(optionally its subnet)
    - `macaddr` -> Mac address

#### UUID type
- UUID stands for `Universally Unique IDentifier`

- Distributed Database
    - Database 1
        - User
            - id: UUID
    - Database 2
        - User
            - id: UUID
- Postgreql provides you with the algorithm to generate these values:
    - `gen_random_uuid()` -> v 13+
    - `uuid_generate_v4()`
        
        
```postgresql
CREATE TABLE test (
    uuid UUID DEFAULT uuid_generate_v4(),
    name VARCHAR(100)
);

-- CREATE EXTENSION
CREATE EXTENSION IF NOT EXISTS "uuid-ossp";
```


#### Arrays
- Defines a variable-length multidimentional arrays
- We can create it in two ways
    - Build-in `ARRAY` construct
    - array-literal syntax
    
```postgresql
CREATE TABLE test (
    name TEXT,
    items INT ARRAY, -- INT[]
    square INT[2][2] -- INT ARRAY[2][2] = [[4,5],[2,5]]
);
--- insert
INSERT INTO test
VALUES
    ('kevin', '{2,3,4,5,6}', ARRAY[[4,5],[2,5]]),
    ('eyong', '{8,9,10}', ARRAY[[1,2],[9,11]]);  --`'{{1,2},{9,11}}'`
    
-- ACCESSING
SELECT * FROM test WHERE 11 = ANY(square[2:2]);

```

##### About Arrays
- They are one-index based
- The slicing doesn't have the `step` property.
- The slicing `stop` property is inclusive.

#### Composite Types
- Represents the structure of a row.

```postgresql
-- CREATE COMPOSITE TYPE
CREATE TYPE name_item AS (
    title VARCHAR(10),
    first_name VARCHAR(100),
    last_name VARCHAR(100)
);

CREATE TABLE person (
    name name_item,
    age SMALLINT
);
-- INSERT
INSERT INTO person 
VALUES 
    (ROW('Mr.','kevin','eyong'), 20),
    ('("Ms.","derrick","eyong")', 30);
    
-- Accessing 
SELECT (colum-name).<composite-name> FROM ..
```

#### Range and Multirange types
- Represents a range of values of some data type.
- For the range of date, postgresql provides `daterange`.

```postgresql
CREATE TABLE test(
    book_title VARCHAR(100),
    read_range DATERANGE -- start date and end date
);
-- INSERT
INSERT INTO test
VALUES
    ('Database for beginners', '[2023-01-01,]'), -- start date, but no end date
    ('Django for prefessionals', '[,]'),          -- no start, no end date
    ('Python for dummies', '[2023-02-01, 2023-02-18]');  -- a start and an end date

```
- Postgresql provides `lower()` and `upper()` to get the lower-bound and upper-bound of the range.

```postgresql
-- Get all rows where the upper bound of date range is null
SELECT * FROM test WHERE upper(read_range) IS NULL;

-- Get all rows with a start date between 2023-01-01 and 2023-03-01
SELECT * FROM test WHERE lower(read_range) BETWEEN '2023-01-01'::DATE AND '2023-03-01'::DATE;

-- Update the lower bound of the read_range.
UPDATE test
SET read_range = DATERANGE('2023-01-11',Null)
WHERE book_title  LIKE '%Django%';

UPDATE test
SET read_range = DATERANGE(lower(read_range), '2023-02-11')
WHERE book_title LIKE '%Django%';

```

#### JSON
- `json`, `jsonb`.

```postgresql
CREATE TABLE test (
    id SERIAL PRIMARY KEY,
    info JSON NOT NULL,
    info2 JSONB NOT NULL
);

-- INSERT
INSERT INTO test(info, info2)
VALUES
    ('{"name" :"oil", "store": {"store_name":"jim", "store_id":5, "store_id":6}}', 
     '{"name":"mango", "store": {"store_name":"ali", "store_id":10, "store_id":12}}');
```
- Postgresql provides two operators `->` and `->>`.
    - `->`: returns JSON object field by key.
    - `->>`: returns JSON object field by text
   
```postgresql
SELECT info -> 'name  ' FROM test;

```

# Postgresql Aggregate Functions

In [1]:
```json
{
    1:2,
    'name':'kevin',
    'family': ['kevin', 'pet'],
    'member': {
        
    }
}
```

SyntaxError: invalid syntax (2741560372.py, line 1)

In [ ]:
import time
import os
def convert(t):##function
    return t * 60
def countdown(t,label):##function
    #60
    while t:
        min, sec =divmod(t,60)###builtin function to give minutes and seconds.
        print(f"{label}: {min:02d}:{sec:02d}", end="\r")##t // 60 then t  % 60
        time.sleep(1)
        t -= 1
def pomodoro(work,rest):##function
    ##convert min to seconds
    w = convert(work)
    r = convert(rest)
    countdown(w,"Work-Time")
    os.system("clear")
    countdown(r,"Break-Time")
    os.system("clear")
    
  
work = int(input("Enter Work-Time (minutes): "))
rest = int(input("Enter Break-Time(minutes): "))
print(work)
print(rest)
pomodoro(work, rest)

foreign key references primary key
- foreign key references `UNIQUE NOT NULL`.

# SQL Functions and Subqueries

### Aggregate Functions (AF)
> Are build-in functions that takes in multiple values and return a single value.

#### Different AF
- COUNT() -> `Returns a count of rows
- MAX() -> `Returns the maximum, or largest row value
- MIN() -> `Returns the minimum, or smallest row value
- AVG() -> `Returns the average of a set of numbers
- SUM() -> `Returns the sum of a set of numbers


**Get the number of rows in the table student**


```postgresql
SELECT COUNT(*) FROM members.student;
```

**Get the max id in the table**

```postgresql
SELECT MAX(id) FROM members.student;
```

**Get the max number of chracters for the first name column**

```postgresql
-- Use a function to get the number of cheracters of each first name 
-- Use MAX function on the result to get the final result.

SELECT MAX(LENGTH(first_name)) FROM members.student;
```

##### Using `GROUP BY` with aggregate function

```
SELECT COUNT(*), gender 
FROM members.student
GROUP BY gender;
```

**How many students there are of each gender and title**

```postgresql
-- Follow the rule of thumb
    -- If you have aggregates and non-aggregates, the non-aggregates should be mentioned in the GROUP BY clause.

SELECT COUNT(*), gender, title
FROM members.student
GROUP BY gender, title;
```

- When to filter
    - Filter before the grouping
        - WHERE clause
    - Filter after the grouping
        - HAVING clause
        
```postgresql
SELECT COUNT(*), gender, title
FROM members.student
-- WHERE title IS NOT NULL
GROUP BY gender, title
HAVING title IS NOT NULL;

```

```postgresql
SELECT COUNT(*) all_student, gender, title
FROM members.student
WHERE title IS NOT NULL
GROUP BY gender, title
HAVING COUNT(*) > 90;
```

### Exercise
- Use `||`, `AGE()`,`EXTRACT()` and `CASE expression` to find the age of each students, using their date of birth.

```postgresql

full_name         |     age
----------------------------
kevin eyong       |  1 years
kevin enow        |  1 months
kenz gob          |  15 days
```


```postgresql
SELECT first_name || ' ' || last_name full_name,
CASE WHEN EXTRACT(YEAR FROM AGE(NOW(), date_of_birth)) > 0
    THEN EXTRACT(YEAR FROM AGE(NOW(), date_of_birth)) || ' ' || 'Year(s)'
    WHEN EXTRACT(MONTH FROM AGE(NOW(), date_of_birth)) > 0
    THEN EXTRACT(MONTH FROM AGE(NOW(), date_of_birth)) || ' ' || 'Month(s)'
    ELSE EXTRACT(DAY FROM AGE(NOW(), date_of_birth)) || ' ' || 'Day(s)'
END AS "age"
FROM members.student
WHERE date_of_birth IS NOT NULL;
```

### Other Functions
- abs()
- upper()
- initcap()
- substr()
- left()
- right()

## Postgresql Subqueries
> A subquery is a query that is nested within another query

```postgresql

OUTER QUERY
    ( INNER QUERY);
```

### Some important points
- The subquery is also known as `inner query` while the query enclosing it is called `outer query`.
- The subquery usually run first.
- The subquery should be enclosed in parentheses.
- Suquery doesn't end with semi colon
- The subquery may need to have an alias.
- The ruturn value of the subquery should depend on the outer query.


```postgresql
-- Return the two oldest students full names.

SELECT first_name || ' ' || last_name full_name
FROM members.student
WHERE date_of_birth IS NOT NULL
ORDER BY date_of_birth ASC
LIMIT 2;


-- Return the two oldest students

SELECT UPPER(first_name)
FROM members.student
WHERE date_of_birth IS NOT NULL
ORDER BY date_of_birth ASC
LIMIT 2;

--
SELECT UPPER(SUBSTRING(first_name, 1,2)) first_name
FROM members.student
WHERE date_of_birth IS NOT NULL
ORDER BY date_of_birth ASC
LIMIT 2;

```


```postgresql
-- Normal query
SELECT first_name || ' ' || last_name full_name



-- subquery

SELECT first_name, last_name
FROM members.student
WHERE date_of_birth IS NOT NULL
ORDER BY date_of_birth ASC
LIMIT 2;


--- First 

SELECT first_name || ' ' || last_name full_name
FROM 
(
    SELECT first_name, last_name
    FROM members.student
    WHERE date_of_birth IS NOT NULL
    ORDER BY date_of_birth ASC
    LIMIT 2
 )x;
 
 -- Second
 
SELECT UPPER(first_name)
FROM 
(
    SELECT first_name, last_name
    FROM members.student
    WHERE date_of_birth IS NOT NULL
    ORDER BY date_of_birth ASC
    LIMIT 2
 )x;
 
-- Third

--
SELECT UPPER(SUBSTRING(first_name, 1,2)) first_name
FROM 
(
    SELECT first_name, last_name
    FROM members.student
    WHERE date_of_birth IS NOT NULL
    ORDER BY date_of_birth ASC
    LIMIT 2
 )x;
```

```postgresql

CREATE VIEW members.student_view AS
(
    SELECT first_name, last_name
    FROM members.student
    WHERE date_of_birth IS NOT NULL
    ORDER BY date_of_birth ASC
    LIMIT 2
);

--- First 

SELECT first_name || ' ' || last_name full_name
FROM members.student_view;

```

### WHERE clause
- EXISTS, IN, NOT IN, '!= <> ='

```postgresql
SELECT 'There is an age threshold alert' age_alert
FROM members.student
WHERE id > (
    select id 
    from members.student
    where extract(year from age(now(), date_of_birth)) = 1
    limit 1
    );

```

#### Few things to note
- The equal and not equal to operators only work with one columned row or one value
- The `IN` operator supports only one column, but may have multiple rows
- The `EXISTS` supports all.

## Self-Study
- Corelated Subquery

```postgresql

SELECT first_name
FROM tesla.employee e
WHERE e.emp_id = (
    SELECT emp_id
    FROM tesla.emp_role ep
    WHERE e.emp_id = ep.emp_id
    AND salary = 500
    );
```

## Database Consistency
- It comes in two flavors:
    - Strong consistency
    - Week consistency

## Database Transaction

> A database transaction is used to **affect the state of data** in the database through `INSERT, UPDATE, DELETE` or `SELECT` queries as a **single unit of work**, so that all of them **pass together**, or **fail together**.

- If it fails
    - The changes before the point of failure will be rollback.
    
```python

def test():
    print('hello world')
    db.write('400')
    print('okay done')
    db.write('500') # line of code fails
```           
  
- Registration
    - user_login
        - username
        - password
    - user_info
        - user_id
        - balance
        
## ACID quarantees

- `Atomicity`: A database transaction must be done as whole or not done at all.
- `Consistency`: Tries to maintain the database state integrity base on the set rules
- `Isolation`: Each transaction is independent of other transactions.
- `Durability`: All transaction results are permanently preserved.


- All relational databases can provide `ACID quarantees`.

## BASED quarantees


## Time to Build
- We are going to build a simple app called `PayAsYouGo`
- Create a database called `payasyougo`
- Then create a script for the sql queries
- Then execute in that database. `\i <path-to-script>`

### Tables
- user_login
    - id `SERIAL PRIMARY KEY`
    - username `VARCHAR(100) NOT NULL UNIQUE` 
    - password  `VARCHAR(200) NOT NULL`
    - created_at `TIMESTAMP DEFAULT current_timestamp`
- user_info
    - user_id `INT REFERENCES user_login (id)`
    - balance `NUMERIC` check to make sure balance is >= 100
    - created_at `TIMESTAMP  DEFAULT current_timestamp`
- transaction
    - user_id `INT REFERENCES user_login (id)`
    - before_balance `NUMERIC NOT NULL`
    - after_balance `NUMERIC NOT NULL`
    - diff_balance `NUMERIC GENERATED ALWAYS AS (after_balance - before_balance) STORED`
    - transaction_type trans_type ## ENUM('CREDIT', 'DEBIT')
    - created_at `TIMESTAMP  DEFAULT current_timestamp`

## Transactions
- A transaction starts with `BEGIN;` and may end with either `COMMIT;` or `ROLLBACK`;

```postgresql
-- syntax

BEGIN;
-- SQL QUERIES
COMMIT;
```
### Register users
```postgresql
-- We need this exception to be able to work with gen_salt and crypt
CREATE EXTENSION pgcrypto;
```

```postgresql
-- Register a user
BEGIN;
    
    WITH insert_user_login AS (
        -- fill user_login table
        INSERT INTO user_login(username, password)
        VALUES('kevin', crypt('eyong2023', gen_salt('md5'))) RETURNING id
    )
    
        -- fill the user_info table
        INSERT INTO user_info(user_id, balance)
        SELECT id, 100 FROM insert_user_login;
    
COMMIT;
    

```

```postgresql
-- VERIFY USER PASSWORD

SELECT * FROM user_login WHERE password = crypt('<user-password>', password);

```

## Exercises
- Complete the app by completing the following;

### Credit and Debit a user's account
- CREDIT
    - Credit the account by updating in the user_info table
    - track the transaction in the transaction table.
- DEBIT
    - Check that the account to debit has sufficient balance
    - Debit the account by updating the user_info table
    - Track the transaction in the transaction table
    
### Account to Account transfer
- Say `user A` wants to transfer money to `user B`
    - Steps:
        - Check `user A` balance to be sure it has sufficient balance
        - Debit from `user A`
        - Credit `user B`
        - Track the transaction in the transaction table.

# Database Concurrency

> Database Concurrency means two or more transactions are modifying the state of data in the database at the same time.

- SQL defines four level of transaction isolation
    - `Dirty read`:
        - A transaction reads data written by a concurrent ucommitted transaction.
    - `Nonrepeatable read`:
        - A transaction re-reads data it has previously read and finds out that the data has been modified by another transaction.
    - `Phantom read`:
        - A transaction re-executes a query returning a set of rows that satisfy a search condition and finds that the set of rows satisfying the condition has changed due to another recently committed transaction.
    - `Serialization`:
         - The result of successfully committing a group of transactions is inconsistent with all possible ordering of running those transactions on at the time.
         
- PostgreSQL defines 
    - `read committed` -> disallows `dirty read`
    - `repeatable read` -> disallows `dirty read`, `nonrepeatable read`, [`phantom read`]
    - `serializable` -> disallows everything.
    

## Default transaction level

### UPDATE

```postgresql
BEGIN; -- READ COMMITTED
    UPDATE user_login
    SET username = 'kenz'
    WHERE id = 1;
```

### Experiment with
- DELETE
- INSERT

### Setting Transaction isolation level

```postgresql

-- BEGIN TRANSACTION ISOLATION LEVEL <isolation-name>

BEGIN TRANSACTION ISOLATION LEVEL serializable;
    UPDATE user_login
    SET username = 'enow'
    WHERE id = 1;
```


### Transactional DDLs

## Understanding Locking in PostgreSQL
- `ACCESS SHARE` -> By default, this is implemented by the `SELECT`
- `ACCESS EXCLUSIVE` -> By default, this is implemented by the `DROP TABLE`.

```postgresql
-- LOCK TABLE <table-name> IN <lock-mode>;

BEGIN; -- isolation level
    LOCK TABLE user_login IN ACCESS EXCLUSIVE MODE;
    SELECT * FROM user_login;
```



### Investigate the database for any lock and wait transaction

```postgresql

SELECT pid, wait_event, wait_event_type, query FROM pg_stat_activity;
```